# Chapter 13 — Prompt Registry: Register, Validate, and Promote

This notebook demonstrates the full prompt deployment lifecycle:
register a versioned prompt template, validate it in staging, and
promote it to production. This mirrors the model validation gate
pattern from `01_model_packaging`.

**Prerequisites:**
- MLflow Prompt Registry enabled in your workspace
- A Unity Catalog schema with CREATE FUNCTION and EXECUTE privileges

In [0]:
%pip install mlflow
%restart_python

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


## 1. Register a Versioned Prompt Template

Prompt templates use double-brace `{{variable}}` syntax for variables.
Each registration under the same name creates a new immutable version
automatically, similar to model versioning in Unity Catalog. The prompt
name must include the full Unity Catalog path (`catalog.schema.name`).

In [0]:
import mlflow

CATALOG = "demo"
SCHEMA = "finance"
PROMPT_NAME = f"{CATALOG}.{SCHEMA}.data_quality_agent_system_prompt"

prompt = mlflow.genai.register_prompt(
    name=PROMPT_NAME,
    template=(
        "You are a data quality analysis agent. Analyze the "
        "{{table_name}} table in {{catalog}}.{{schema}} "
        "and propose data quality rules based on column metadata."
    ),
    commit_message="Initial system prompt for data quality agent",
    tags={"team": "data-engineering"},
)

print(f"Registered prompt version: {prompt.version}")

Registered prompt version: 2


In [0]:
mlflow.genai.set_prompt_alias(
    name=PROMPT_NAME,
    alias="staging",
    version=prompt.version,
)

print(f"Assigned 'staging' alias to version {prompt.version}")

Assigned 'staging' alias to version 2


## 2. Validate the Prompt Before Promoting

Just as we validate models before promoting them, we validate prompts.
A deployment pipeline loads the candidate from the staging alias,
checks that the template contains the expected variables, and promotes
only if the checks pass.

In [0]:
# Load the candidate prompt by alias
prompt = mlflow.genai.load_prompt(f"prompts:/{PROMPT_NAME}@staging")

# Validate the template has the variables the agent expects
required_vars = ["{{table_name}}", "{{catalog}}", "{{schema}}"]
for var in required_vars:
    assert var in prompt.template, f"Missing variable: {var}" # <1>

print(f"Validation passed for version {prompt.version}")

Validation passed for version 2


## 3. Promote to Production

After validation passes, we promote the prompt by assigning the
production alias. No code change or redeployment needed — the
application loads the prompt by alias at runtime.

In [0]:
mlflow.genai.set_prompt_alias(
    name=PROMPT_NAME,
    alias="production",
    version=prompt.version,                                # <2>
)

print(f"Promoted version {prompt.version} to 'production' alias")

Promoted version 2 to 'production' alias


## 4. Load the Production Prompt at Runtime

In a deployed application, we load the prompt by alias rather than
by version number. When we promote a new version, the application
picks it up automatically without redeployment.

In [0]:
production_prompt = mlflow.genai.load_prompt(
    f"prompts:/{PROMPT_NAME}@production"
)

# Render the template with actual values
rendered = production_prompt.format(
    table_name="customer_transactions",
    catalog="prod_catalog",
    schema="finance",
)

print(rendered)

You are a data quality analysis agent. Analyze the customer_transactions table in prod_catalog.finance and propose data quality rules based on column metadata.
